In [1]:
import sys

In [2]:
sys.path.append("/projects/computer-vision/")

In [3]:
import tensorflow as tf
import tensorflow.keras as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

2022-11-30 17:10:43.104214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
gpus = tf.config.list_physical_devices('GPU')

2022-11-30 17:10:44.795356: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-30 17:10:44.798034: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-30 17:10:44.798279: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.


In [5]:
# if gpus:
#     for gpu in gpus:
#         tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])

In [6]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
       tf.config.experimental.set_memory_growth(gpu,True)

In [7]:
from utils import cv_utils

In [8]:
%matplotlib

Using matplotlib backend: <object object at 0x7f7d5e12da80>


In [9]:
import scipy as scp

In [10]:
# Loading data
# Loading metadata
wiki_metadata_df = scp.io.loadmat('/datasets/wiki/wiki.mat')

In [11]:
metadata_columns = wiki_metadata_df['wiki'].dtype.fields.keys()

In [12]:
metadata_df = wiki_metadata_df['wiki'][0][0]

In [13]:
metadata_dict = { col:metadata_df[i] for col, i in zip(metadata_columns, range(len(metadata_columns))) }

In [14]:
[ metadata_dict[c].shape for c in metadata_dict ]

[(1, 62328),
 (1, 62328),
 (1, 62328),
 (1, 62328),
 (1, 62328),
 (1, 62328),
 (1, 62328),
 (1, 62328)]

In [15]:
metadata_df = pd.DataFrame({ c:metadata_dict[c][0].tolist() for c in metadata_dict })

In [16]:
metadata_df.head()

,dob,photo_taken,full_path,gender,name,face_location,face_score,second_face_score
0,723671,2009,[17/10000217_1981-05-05_2009.jpg],1.0,[Sami Jauhojärvi],"[[111.29109473290997, 111.29109473290997, 252....",4.300962,NaN
1,703186,1964,[48/10000548_1925-04-04_1964.jpg],1.0,[Dettmar Cramer],"[[252.48330229530742, 126.68165114765371, 354....",2.645639,1.949248
2,711677,2008,[12/100012_1948-07-03_2008.jpg],1.0,[Marc Okrand],"[[113.52, 169.83999999999997, 366.08, 422.4]]",4.329329,NaN
3,705061,1961,[65/10001965_1930-05-23_1961.jpg],1.0,[Aleksandar Matanović],"[[1, 1, 634, 440]]",-inf,NaN
4,720044,2012,[16/10002116_1971-05-31_2012.jpg],0.0,[Diana Damrau],"[[171.61031405173117, 75.57451239763239, 266.7...",3.408442,NaN


In [17]:
metadata_df['full_path'] = metadata_df['full_path'].apply(lambda x: x[0])

In [18]:
YEAR_ZERO_UNIX_EPOCH = 719529 # epoch of 1970-01-01

In [19]:
metadata_df.describe().T

/envs/cv-env/lib/python3.8/site-packages/numpy/lib/function_base.py:4527: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,count,mean,std,min,25%,50%,75%,max
dob,62328.0,7.166810e+05,10832.627367,4077.000000,710673.000000,719269.000000,724702.250000,736011.000000
photo_taken,62328.0,1.998766e+03,20.907937,1940.000000,1992.750000,2009.000000,2012.000000,2015.000000
gender,59685.0,7.885231e-01,0.408359,0.000000,1.000000,1.000000,1.000000,1.000000
face_score,62328.0,-inf,NaN,-inf,NaN,2.463949,3.760014,7.081268
second_face_score,4096.0,2.009050e+00,0.981277,0.731419,1.164925,1.839065,2.655757,5.463147


In [20]:
metadata_df['dob'].dropna().shape, metadata_df['dob'].shape

((62328,), (62328,))

In [21]:
metadata_df['gender'].value_counts()
# Oh only binary gender!? HOHO !

1.0    47063
0.0    12622
Name: gender, dtype: int64

In [22]:
metadata_df['gender'].dropna().shape, metadata_df['gender'].shape

((59685,), (62328,))

In [23]:
metadata_df = metadata_df.dropna(subset=['gender'])

In [24]:
metadata_df = metadata_df[metadata_df['face_score'].apply(np.isfinite)]

In [25]:
metadata_df['name'] = metadata_df['name'].apply(lambda x: x[0])

In [26]:
metadata_df.head()

,dob,photo_taken,full_path,gender,name,face_location,face_score,second_face_score
0,723671,2009,17/10000217_1981-05-05_2009.jpg,1.0,Sami Jauhojärvi,"[[111.29109473290997, 111.29109473290997, 252....",4.300962,NaN
1,703186,1964,48/10000548_1925-04-04_1964.jpg,1.0,Dettmar Cramer,"[[252.48330229530742, 126.68165114765371, 354....",2.645639,1.949248
2,711677,2008,12/100012_1948-07-03_2008.jpg,1.0,Marc Okrand,"[[113.52, 169.83999999999997, 366.08, 422.4]]",4.329329,NaN
4,720044,2012,16/10002116_1971-05-31_2012.jpg,0.0,Diana Damrau,"[[171.61031405173117, 75.57451239763239, 266.7...",3.408442,NaN
5,716189,2012,02/10002702_1960-11-09_2012.jpg,0.0,Krista Tippett,"[[274.76563240288175, 57.7700900839337, 376.88...",4.748056,NaN


In [27]:
metadata_df['gender'].hist()

<AxesSubplot: >

In [28]:
# We will sample 10K females and 10k males images to balance out the dataset

In [29]:
def from_matlab_to_datetime(d):
    try:
        return pd.to_datetime(d-YEAR_ZERO_UNIX_EPOCH, unit='D')
    except:
        None

In [30]:
metadata_df['date_of_birth'] = metadata_df['dob'].apply(from_matlab_to_datetime)

In [31]:
metadata_df['dob'].iloc[:1].apply(from_matlab_to_datetime)

0   1981-05-05
Name: dob, dtype: datetime64[ns]

In [32]:
metadata_df['date_of_birth'].hist(bins=50)

<AxesSubplot: >

In [33]:
import datetime as dt

In [34]:
metadata_df['year_of_birth'] = metadata_df['date_of_birth'].apply(lambda x : x.year)

In [35]:
metadata_df['age'] = metadata_df['photo_taken'] - metadata_df['year_of_birth']

In [36]:
metadata_df['age'].hist(bins=50)

<AxesSubplot: >

In [37]:
metadata_df.head()

,dob,photo_taken,full_path,gender,name,face_location,face_score,second_face_score,date_of_birth,year_of_birth,age
0,723671,2009,17/10000217_1981-05-05_2009.jpg,1.0,Sami Jauhojärvi,"[[111.29109473290997, 111.29109473290997, 252....",4.300962,NaN,1981-05-05,1981.0,28.0
1,703186,1964,48/10000548_1925-04-04_1964.jpg,1.0,Dettmar Cramer,"[[252.48330229530742, 126.68165114765371, 354....",2.645639,1.949248,1925-04-04,1925.0,39.0
2,711677,2008,12/100012_1948-07-03_2008.jpg,1.0,Marc Okrand,"[[113.52, 169.83999999999997, 366.08, 422.4]]",4.329329,NaN,1948-07-03,1948.0,60.0
4,720044,2012,16/10002116_1971-05-31_2012.jpg,0.0,Diana Damrau,"[[171.61031405173117, 75.57451239763239, 266.7...",3.408442,NaN,1971-05-31,1971.0,41.0
5,716189,2012,02/10002702_1960-11-09_2012.jpg,0.0,Krista Tippett,"[[274.76563240288175, 57.7700900839337, 376.88...",4.748056,NaN,1960-11-09,1960.0,52.0


In [38]:
import cv2 as cv
import utils

In [39]:
SAMPLE_SIZE = 8000

In [40]:
metadata_df['full_path'].dropna().shape,metadata_df['full_path'].shape, metadata_df['full_path'].drop_duplicates().shape

((43452,), (43452,), (43452,))

In [41]:
metadata_df.index = metadata_df['full_path']

In [42]:
female_index = np.random.choice(metadata_df[metadata_df['gender'] == 0].index, SAMPLE_SIZE, replace=False)

In [43]:
male_index = np.random.choice(metadata_df[metadata_df['gender'] == 1].index, SAMPLE_SIZE, replace=False)

In [44]:
dataset = pd.concat([
    metadata_df.loc[female_index],
    metadata_df.loc[male_index]
])

In [45]:
print(dataset.shape)
dataset.head()

(16000, 11)


,dob,photo_taken,full_path,gender,name,face_location,face_score,second_face_score,date_of_birth,year_of_birth,age
full_path,,,,,,,,,,,
65/23465465_1956-08-03_2007.jpg,714630,2007,65/23465465_1956-08-03_2007.jpg,0.0,Susan Fernandez,"[[169.83999999999997, 43.12, 295.6799999999999...",2.768081,NaN,1956-08-03,1956.0,51.0
36/25824136_1988-03-05_2010.jpg,726167,2010,36/25824136_1988-03-05_2010.jpg,0.0,Kiersten Dallstream,"[[239.6653855760832, 83.2722019227873, 312.786...",3.923385,NaN,1988-03-05,1988.0,22.0
11/15035211_1942-08-23_1967.jpg,709536,1967,11/15035211_1942-08-23_1967.jpg,0.0,Patricia McBride,"[[151.78570352511005, 91.42342211506603, 286.7...",4.813193,NaN,1942-08-23,1942.0,25.0
87/22350387_1925-03-13_1949.jpg,703164,1949,87/22350387_1925-03-13_1949.jpg,0.0,Susan Douglas Rubes,"[[74.42411158725653, 147.96822317451307, 294.1...",3.691288,NaN,1925-03-13,1925.0,24.0
24/2410224_1950-01-29_1988.jpg,712252,1988,24/2410224_1950-01-29_1988.jpg,0.0,Ann Jillian,"[[61.26358993209154, 101.76331655348591, 242.9...",3.314276,NaN,1950-01-29,1950.0,38.0


In [46]:
input_shape = (2, 28, 28, 3)
x = np.arange(np.prod(input_shape)).reshape(input_shape)

In [47]:
x.shape

(2, 28, 28, 3)

In [48]:
# [ (t, b), (l, r) for t, l, b, r in tdataset['face_location'][0][0]

In [49]:
# dataset['crop_coords'] = dataset['face_location'][0]

In [50]:
dataset['full_path'].apply(lambda x: x[-3:]).value_counts()

jpg    16000
Name: full_path, dtype: int64

We only have image of format `jpg`

In [51]:
# ROOT_PATH = "/datasets/wiki/images/"
ROOT_PATH = "/datasets/wiki_crop/"

In [52]:
# images = tf.keras.preprocessing.image_dataset_from_directory(ROOT_PATH, interpolation='gaussian')

In [53]:
IMG_HEIGHT, IMG_WIDTH = 256, 256

In [54]:
tf_dataset_df = tf.data.Dataset.from_tensor_slices(
    (
        tf.cast(dataset['full_path'], tf.string),
        # tf.cast(dataset['crop_coords'], tf.int32),
        tf.cast(dataset['age'], tf.int32),
        tf.cast(dataset['gender'], tf.int32)
    )
)

2022-11-30 17:10:55.110414: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 17:10:55.111216: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-30 17:10:55.111462: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-30 17:10:55.111692: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_nod

In [55]:
for f, a, d in tf_dataset_df.take(1):
    print(f)

tf.Tensor(b'65/23465465_1956-08-03_2007.jpg', shape=(), dtype=string)


In [56]:
# we use the path as key in this dataset
def decode_img(img):
    img = tf.io.decode_jpeg(img, channels=3)
    return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

def preprocess_data(path, age, label):
    img = tf.io.read_file(ROOT_PATH + path)
    img = decode_img(img)
    return img, age

In [57]:
TRAIN_SIZE, VAL_SIZE, TEST_SIZE = 0.8, 0.1, 0.1

In [58]:
df_size = len(tf_dataset_df)

In [59]:
tf_dataset_df = tf_dataset_df.shuffle(buffer_size=df_size)
train_df = tf_dataset_df.take(round(TRAIN_SIZE * df_size))
test_df = tf_dataset_df.skip(round(TRAIN_SIZE * df_size))
val_df = test_df.take(round(VAL_SIZE * df_size))
test_df = test_df.skip(round(VAL_SIZE * df_size))

In [60]:
AUTOTUNE = tf.data.AUTOTUNE
STANDARD_BATCH_SIZE, STANDARD_PREFETCH_BUFFER = 5, 5

In [61]:
for f, a, d in train_df.take(1):
    print(f)

tf.Tensor(b'76/2431776_1982-01-28_2014.jpg', shape=(), dtype=string)


In [62]:
train_df = train_df.map(preprocess_data, num_parallel_calls=AUTOTUNE)
test_df = test_df.map(preprocess_data, num_parallel_calls=AUTOTUNE)
val_df = val_df.map(preprocess_data, num_parallel_calls=AUTOTUNE)

In [63]:
def prepare_batch_load(tf_df, batch_size=STANDARD_BATCH_SIZE, prefetch_buffer=STANDARD_PREFETCH_BUFFER):
    tf_df = tf_df.shuffle(buffer_size=(len(tf_df)))
    tf_df = tf_df.batch(batch_size)
    tf_df = tf_df.prefetch(buffer_size=prefetch_buffer)
    return tf_df

In [64]:
train_df = prepare_batch_load(train_df)
val_df = prepare_batch_load(val_df)
test_df = prepare_batch_load(test_df)

In [65]:
class SimpleConvModel(K.Model):
    def __init__(self, filters, kernel_size, strides, pool_size, activation, n_units=1):
        super(SimpleConvModel, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.norm = K.layers.Rescaling(1./255)
        # Defining the layers
        self.conv2D_0 = K.layers.Conv2D(
            name='conv2D_0', 
            filters=self.filters, kernel_size=self.kernel_size, activation='relu', padding='same'
        )

        self.max_pool = K.layers.MaxPool2D(
            name='max_pool_0',
            pool_size=pool_size, strides=strides
        )

        self.flatten = K.layers.Flatten()

        self.dense_0 = K.layers.Dense(units=32, activation='relu', name='dense_0')
        self.predictor = K.layers.Dense(units=n_units, activation=K.activations.get(activation), name='age_predictor')
        
    def __call__(self, inputs, training=True):
        x = self.norm(inputs)
        x = self.conv2D_0(x)
        x = self.max_pool(x)
        x = self.flatten(x)
        x = self.dense_0(x)
        x = self.predictor(x)
        return x

In [66]:
simple_model = SimpleConvModel(filters=64, kernel_size=3, strides=1, pool_size=2, activation='linear')

In [67]:
simple_model.compile(
    optimizer='adam', loss='mse', metrics=['mape', 'mae']
)

In [68]:
# [ c[0].numpy() for c in train_df.take(1)]

In [69]:
simple_model.fit(train_df, validation_data=val_df, epochs=10)

Epoch 1/10


2022-11-30 16:53:50.003580: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2022-11-30 16:53:51.057232: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1ea0e9f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-11-30 16:53:51.057294: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2022-11-30 16:53:51.062026: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-11-30 16:53:51.174693: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3158/3200 [============================>.] - ETA: 0s - loss: 622.3773 - mape: 43.6536 - mae: 14.5364

KeyboardInterrupt: 

In [ ]:
simple_model.evaluate(test_df)

2022-11-30 14:18:28.016614: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-11-30 14:18:28.016706: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: cHRM chunk does not match sRGB


160/160 [==============================] - 2s 8ms/step - loss: 898.7798 - mape: 63.5435 - mae: 25.1694


[898.77978515625, 63.54345703125, 25.169401168823242]

In [65]:
class VGGBlock(K.Model):
    def __init__(self, filters, kernel_size, repetitions, strides=2, pool_size=2):
        super(VGGBlock, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        # self.norm = K.layers.Rescaling(1./255)


        self.repetitions = repetitions

        for rep in range(self.repetitions):
            vars(self)[f'conv2D_{rep}'] = K.layers.Conv2D(
                filters=filters, kernel_size=kernel_size, activation='relu', padding='same'
            )

        # Defining the layers
        # self.conv2D_0 = K.layers.Conv2D(
        #     name='conv2D_0', 
        #     filters=self.filters, kernel_size=self.kernel_size, activation='relu', padding='same'
        # )

        self.max_pool = K.layers.MaxPool2D(
            pool_size=pool_size, strides=strides
        )
        
    def __call__(self, inputs, training=True):
        x = self.conv2D_0(inputs)
        x = self.max_pool(x)
        return x

In [66]:
class VGG(K.Model):
    def __init__(self, activation):
        super(VGG, self).__init__()
        self.activation = K.activations.get(activation)
        self.norm = K.layers.Rescaling(1./255)
        self.block_0 = VGGBlock(64, 3, 2)
        self.block_1 = VGGBlock(128, 3, 2)
        self.block_2 = VGGBlock(256, 3, 3)
        self.block_3 = VGGBlock(512, 3, 3)
        self.block_4 = VGGBlock(512, 3, 3)
        self.flatten = K.layers.Flatten()
        self.dense = K.layers.Dense(units=4096, activation='relu')
        self.predictor = K.layers.Dense(units=1, activation=self.activation)

    def __call__(self, inputs, training=True):
        x = self.norm(inputs)
        x = self.block_0(x)
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.block_4(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.predictor(x)
        return x 

In [67]:
vgg_model = VGG('linear')

In [68]:
vgg_model.compile(
    optimizer='adam', loss='mse', metrics=['mae', 'mape']
)

In [69]:
vgg_model.fit(
    train_df,
    validation_data=val_df,
    epochs = 10
)

Epoch 1/10


2022-11-30 17:11:47.360276: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 7416 of 12800
2022-11-30 17:11:54.747772: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.
2022-11-30 17:11:55.341778: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2022-11-30 17:11:56.748096: W tensorflow/tsl/framework/bfc_allocator.cc:360] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2022-11-30 17:11:57.104783: I tensorflow/compiler/xla/service/service.cc:173] XLA 

2560/2560 [==============================] - 123s 40ms/step - loss: 360287962136576.0000 - mae: 167774.2812 - mape: 6438675.0000 - val_loss: 238.8466 - val_mae: 11.9667 - val_mape: 26456886.0000
Epoch 2/10


2022-11-30 17:13:50.066028: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 10841 of 12800


   3/2560 [..............................] - ETA: 1:28 - loss: 203.0317 - mae: 11.6412 - mape: 27.5319  

...

2022-11-30 17:17:37.088246: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 12595 of 12800
2022-11-30 17:17:37.251903: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


2560/2560 [==============================] - 110s 39ms/step - loss: 360287962136576.0000 - mae: 167791.7344 - mape: 3061576.2500 - val_loss: 712.0352 - val_mae: 20.8333 - val_mape: 64.6326
Epoch 5/10
2560/2560 [==============================] - 110s 39ms/step - loss: 360287962136576.0000 - mae: 167811.5625 - mape: 11008714.0000 - val_loss: 1854.8348 - val_mae: 34.0012 - val_mape: 101.1152
Epoch 6/10
2560/2560 [==============================] - 119s 43ms/step - loss: 360287928582144.0000 - mae: 167871.0781 - mape: 11860108.0000 - val_loss: 13576.5312 - val_mae: 92.7201 - val_mape: 277.8386
Epoch 7/10


2022-11-30 17:23:16.639183: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 12665 of 12800
2022-11-30 17:23:16.749523: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


 183/2560 [=>............................] - ETA: 1:39 - loss: 12092.6641 - mae: 88.0639 - mape: 267.5892

KeyboardInterrupt: 

In [1]:
vgg_model.evaluate(train_df)
vgg_model.evaluate(val_df)
vgg_model.evaluate(test_df)

NameError: name 'vgg_model' is not defined

In [101]:
vgg_model.predict(val_df.take(1))

2022-11-30 14:59:39.016692: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-11-30 14:59:39.181689: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-11-30 14:59:39.235726: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-11-30 14:59:39.435401: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG


1/1 [==============================] - 1s 746ms/step


2022-11-30 14:59:39.514409: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


array([[38.877346],
       [30.025137],
       [36.559353],
       [35.593525],
       [36.950672]], dtype=float32)

In [100]:
list(val_df.take(1))

2022-11-30 14:59:23.091790: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
2022-11-30 14:59:23.173410: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
2022-11-30 14:59:23.559055: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-11-30 14:59:23.587835: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


[(<tf.Tensor: shape=(5, 256, 256, 3), dtype=float32, numpy=
  array([[[[254.       , 255.       , 249.71289  ],
           [253.92516  , 255.       , 249.81383  ],
           [253.40599  , 255.       , 250.203    ],
           ...,
           [251.2517   , 245.82591  , 159.95651  ],
           [252.42282  , 248.17218  , 160.12234  ],
           [253.51562  , 250.0562   , 161.63042  ]],
  
          [[253.94952  , 254.13867  , 244.7943   ],
           [253.31445  , 254.29007  , 246.91898  ],
           [254.05742  , 255.       , 249.48906  ],
           ...,
           [248.75508  , 244.75508  , 157.75508  ],
           [252.61133  , 250.03117  , 164.45102  ],
           [254.89093  , 254.4964   , 169.21906  ]],
  
          [[253.16199  , 255.       , 238.93945  ],
           [253.6707   , 255.       , 241.44882  ],
           [253.56445  , 255.       , 245.10025  ],
           ...,
           [245.36978  , 243.24219  , 159.36848  ],
           [249.9379   , 248.76212  , 165.92265  ],


In [4]:
# ### Regularization in a NN
# J(wi, bi, ...) = 1 / m sum(loss(y_hat_i, y_i)) + [ L2normReg(w[l]) for l in layers ],  
# Frobenius norm (not called L2 for maths reason (linear algrebra)),   
# dw gets a new term = lambda / m.  
# Weight Decay: Multiplying the weight matrix by a (1 -alpha * lambda / 2m)y

# the term lambda forces W close to zero which makes some neurons have a smaller effect in the prediction. L1 norm would actually make them zero